In [ ]:
# required libraries
import os
import pandas as pd
import numpy as np
from pathlib import Path
# Load the required libraries 
from rdflib import Graph, Literal, RDF, URIRef, Namespace
# rdflib knows about some namespaces, like FOAF 
from rdflib.namespace import FOAF, XSD 
# CHECK DATE 
import datetime

In [ ]:
path = str(Path(os.path.abspath(os.getcwd())).parent.absolute())
print(path)

#C:\Users\irfan\Downloads\DB2\
coursesURL = 'C:/Users/irfan/Downloads/DB2/dataset/Courses.csv'

instructorsURL = 'C:/Users/irfan/Downloads/DB2/dataset/Instructors.csv'
reviewURL = 'C:/Users/irfan/Downloads/DB2/dataset/Reviews.csv'
subscribersURL = 'C:/Users/irfan/Downloads/DB2/dataset/Subscribers.csv'
# df = pd.read_csv("C:/Users/irfan/Downloads/DB2/dataset/Departments.csv", usecols = ['department_id'])
# print(df)

departmentsURL = 'C:/Users/irfan/Downloads/DB2/dataset/Departments.csv'

print("File paths are working perfectly.")
# saving folder
savePath = 'C:/Users/irfan/Downloads/DB2/dataset/savefiles/'


In [ ]:
# Load the CSV files in memory
courses = pd.read_csv(coursesURL, sep=',', index_col='CourseID')
courses.dtypes

In [ ]:
courses.info()

In [ ]:
# Load the required libraries
from rdflib import Graph, Literal, RDF, URIRef, Namespace
# rdflib knows about some namespaces, like FOAF
from rdflib.namespace import FOAF, XSD

In [ ]:
# Construct the courses ontology namespaces not known by RDFlib
CO = Namespace("http://www.semanticweb.org/user/ontologies/2022/10/udemy-courses#Course")
DPS = Namespace("http://www.semanticweb.org/user/ontologies/2022/10/udemy-courses#Deparment")


In [ ]:
#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("deparments", DPS)
g.bind("courses", CO)

In [ ]:
# CHECK DATE 
import datetime

In [ ]:
%%time 
#measure execution time

#iterate over the courses dataframe
for index, row in courses.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the course id as URI
    Course = URIRef(CO[index])
    # Add triples using store's add() method.
    g.add((Course, RDF.type, CO.Course))
    g.add((Course, CO['Course_Title'], Literal(row['courseTitle'], datatype=XSD.string)))
    g.add((Course, CO['Course_Price'], Literal(row['price'], datatype=XSD.integer)))

    ## handle department
    #there can be more than one department per course
    for c in str(row['department_name']).split(','):
        cName = c.strip() #strip() method removes any leading (spaces at the beginning) and railing (spaces at the end) characters (space is the default leading character to remove)
        # check if the department exists
        if((departments.index == cName).any() == True):
            #get the department ID, convert to string and get the lower case to match the department ID in the contology 
            dep_id = str(departments[departments.index == cName]['department_id'][0])
            # create the RDF node
            Department = URIRef(DPS[dep_id])
            # add the edge connecting the Course and the Department 
            g.add((Course, CO['hasDepartment'], Department))



In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
savePath = 'C:/Users/irfan/Downloads/DB2/dataset/savefiles/'
with open(savePath + 'courses.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

In [ ]:
# Load the CSV files in memory
inst = pd.read_csv(instructorsURL, sep=',', index_col='instructor_id', keep_default_na=False, na_values=['_'])
rev = pd.read_csv(reviewURL, sep=',', index_col='rev_id', keep_default_na=False, na_values=['_'])
subs = pd.read_csv(subscribersURL, sep=',', index_col='subscriber_id', keep_default_na=False, na_values=['_'])

In [ ]:
print("Print data of instructor.")
inst.info()
print("---------------------------.")
print("Print data of reveiwer.")
rev.info()
print("---------------------------.")
print("Print data of Subscriber.")
subs.info()

In [ ]:
#create a new graph
g = Graph()

In [ ]:
%%time 
#measure execution time

#iterate over the person dataframe
for index, row in inst.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the instructor id as URI
    person = URIRef(CO[index])
    g.add((person, RDF.type, FOAF.person))
    # Add triples using store's add() method.
    g.add((person, FOAF['name'], Literal(row['name'], datatype=XSD.string)))
    if row['date_of_birth'] != '':
        try:
            datetime.datetime.strptime(str(row['date_of_birth']), '%Y-%m-%d')
            g.add((person, CO['birthday'], Literal(row['date_of_birth'], datatype=XSD.date)))
        except ValueError:
            # probably it's the year alone
            # check length
            if (len(row['date_of_birth'])==4):
                #it is a year
                g.add((person, CO['birthday'], Literal(row['date_of_birth']+"-01-01", datatype=XSD.date)))
    
    if row['place_of_birth'] != '':
        g.add((person, CO['birthplace'], Literal(row['place_of_birth'], datatype=XSD.string)))
    
    # check if the death day is not empty--i.e., the person is still alive
#     if row['date_of_death'] != '':
#         try:
#             datetime.datetime.strptime(str(row['date_of_death']), '%Y-%m-%d')
#             g.add((Person, CO['deathDay'], Literal(row['date_of_death'], datatype=XSD.date)))
#         except ValueError:
#             # probably it's the year alone
#             # check length
#             if (len(row['date_of_death'])==4):
#                 #it is a year
#                 g.add((Person, CO['deathDay'], Literal(row['date_of_death']+"-01-01", datatype=XSD.date)))
        
# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("departments", DPS)
g.bind("courses", CO)

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'instroctur.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

In [ ]:
# Load the CSV files in memory
join = pd.read_csv(departmentsURL, sep=',', index_col='department_id', keep_default_na=False, na_values=['_'])

In [ ]:
#create a new graph
g = Graph()

In [ ]:
#regular expressions
import re
dept_name = re.compile('act*')

In [ ]:
join.info()

In [ ]:
print(join)

In [ ]:
%%time 
#measure execution time

#iterate over the join table dataframe
for index, row in join.iterrows():
    # Create the node about the course
    # note that we do not add this resource to the database (created before)
    Course = URIRef(CO[index])
    
    # Create the node about the department
    # note that we do not add this resource to the database (created before)
     #inst = URIRef(CO[row['instructor_id']])
     #inst = CO(row['instructor_id'])
    # get the role of the department
    dept_name = row['department_name']
    
    # we have an actor or actress
    if Web_Development.match(dept_name): 
        g.add((inst, CO['teachs'], Course))
    elif Bussines_Finance.match(dept_name): 
        g.add((inst, CO['teachs'], Course))
    elif Graphic_Design.match(dept_name): 
        g.add((inst, CO['teachs'], Course))
    elif (dept_name=='Musical_Instrument'):
        g.add((inst, CO['teachs'], Course))
    else:
        # note that, with the defined ontology, we cannot caracterize the specifi role of this person in the movie. 
        # why?
        g.add((inst, CO['workedIn'], Course))

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("departments", DPS)
g.bind("courses", CO)

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath1 + 'name_movie_join.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

In [ ]:
# queries
courses = pd.read_csv(coursesURL)
departments = pd.read_csv(departmentsURL)
intstructor = pd.read_csv(instructorsURL)
subscriber = pd.read_csv(subscribersURL)
reveiwer = pd.read_csv(reviewURL)

In [ ]:
# querey 1
courses.head(10)

In [ ]:
# query 2
total_rows=len(courses.axes[0]) #===> Axes of 0 is for a row
total_cols=len(courses.axes[1]) #===> Axes of 0 is for a column
print("Number of Rows: "+str(total_rows))
print("Number of Columns: "+str(total_cols))

In [ ]:
# query 3
result = pd.merge(courses,departments[["department_id","number_of_instructors","number_of_courses"]],on="department_id", how='left')
result.head(20)

In [ ]:
# query 4
result= pd.merge(departments,courses,on='department_id',how='inner',indicator=True)
result.head()

In [ ]:
# query 5
courses.nlargest(10, ['numberOfReviews'])

In [ ]:
# query 6
# creating bool series False for NaN values
bool_series = pd.isnull(courses["numberOfReviews"])
 
# displayed data only with numberOfReviews = NaN
courses[bool_series]

In [ ]:
# query 7
print("Sum: ",courses["price"].sum()) 
print("Mean: ",courses["price"].mean())
print("Maximum: ",courses["price"].max())
print("Minimum: ",courses["price"].min())

In [ ]:
# query 8
# print all the data teacher who is born in italy
print("Contents in csv file:", intstructor)
plt.plot(intstructor.name, intstructor.surname)
plt.show()

In [ ]:
# query 9

bool_series = (reveiwer["rating"] > 3)
 
# displayed data only greater then 3 rating
reveiwer[bool_series]

In [ ]:
# query 10
courses.groupby(["department_name","CourseID"]).first(all)